In [30]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.linalg import svdvals

In [25]:
temp = np.array([1,2,3,3,4])
np.where(temp == 3)[0]

array([2, 3], dtype=int64)

## Rank Guess

In [206]:
def guess_rank(X, nnz):
    maxiter = 1e4
    n = X.shape[0]
    m = X.shape[1]
    epsilon = nnz/np.sqrt(m*n)
    S0 = svdvals(X)
    
    nsval0 = len(S0)
    S1 = S0[0:(nsval0-1)] - S0[1:nsval0]
    nsval1 = len(S1)
    if(nsval1 > 10):
        S1_ = S1/np.mean(S1[(nsval1-10):nsval1])
    else:
        S1_ = S1/np.mean(S1[0:nsval1])
    r1 = 0
    lam = 0.05
    
    itcounter = 0
    while(r1 <= 0):
        itcounter += 1
        #print(len(S1_))
        cost = np.zeros(len(S1_))
        for idx in range(len(S1_)):
            #print(idx)
            cost[idx] = lam*max(S1_[idx:]) + (idx+1)
        #print(cost)
        v2 = min(cost)
        i2 = np.where(cost == v2)[0]
        if(len(i2) == 1):
            r1 = i2
        else:
            r1 = max(i2)
        lam += 0.05
        if(itcounter > maxiter):
            break
    
    if(itcounter <= maxiter):
        cost2 = np.zeros((len(S0) - 1))
        for idx in range(len(S0) - 1):
            cost2[idx] = (S0[idx + 1] + np.sqrt(idx*epsilon)*S0[1]/epsilon)/S0[idx]
        v2 = min(cost2)
        i2 = np.where(cost2 == v2)[0]
        if(len(i2) == 1):
            r2 = i2
        else:
            r2 = max(i2)

        if(r1 > r2):
            r = r1
        else:
            r = r2
        return r
    else:
        r = min(X.shape[0], X.shape[1])
        return r

In [243]:
guess_rank(X, 3)

2

In [204]:
X = np.array([[22,34,74], [53, 64, 96]])

In [165]:
nnz = 3
maxiter = 1e4
n = X.shape[0]
m = X.shape[1]
epsilon = nnz/np.sqrt(m*n)
S0 = svdvals(X)

In [166]:
S0

array([222.61700179,  34.61356819,  28.7512687 ,  18.1365917 ])

In [192]:
nsval0 = len(S0)
S1 = S0[0:(nsval0-1)] - S0[1:nsval0]
nsval1 = len(S1)
if(nsval1 > 10):
    S1_ = S1/np.mean(S1[(nsval1-10):nsval1])
else:
    S1_ = S1/np.mean(S1[0:nsval1])
r1 = 0
lam = 0.05

In [193]:
S1_

array([2.75826081, 0.08600774, 0.15573145])

In [184]:
lam*max(S1_[1:])+1

468.24885985833254

In [195]:
itcounter = 0
print(len(S1_))
while(r1 <= 0):
    itcounter += 1
    cost = np.zeros(len(S1_))
    for idx in range(len(S1_)):
        #print(idx)
        cost[idx] = lam*max(S1_[idx:]) + (idx+1)
    print(cost)
    v2 = min(cost)
    i2 = np.where(cost == v2)[0]
    if(len(i2) == 1):
        r1 = i2
    else:
        r1 = max(i2)
    lam += 0.05
    if(itcounter > maxiter):
        break
print(cost)

3
[1.13791304 2.00778657 3.00778657]
[1.27582608 2.01557315 3.01557315]
[1.41373912 2.02335972 3.02335972]
[1.55165216 2.03114629 3.03114629]
[1.6895652  2.03893286 3.03893286]
[1.82747824 2.04671944 3.04671944]
[1.96539128 2.05450601 3.05450601]
[2.10330432 2.06229258 3.06229258]
[2.10330432 2.06229258 3.06229258]


In [197]:
if(itcounter <= maxiter):
    cost2 = np.zeros((len(S0) - 1))
    for idx in range(len(S0) - 1):
        cost2[idx] = (S0[idx + 1] + np.sqrt(idx*epsilon)*S0[1]/epsilon)/S0[idx]
    v2 = min(cost2)
    i2 = np.where(cost2 == v2)[0]
    if(len(i2) == 1):
        r2 = i2
    else:
        r2 = max(i2)
        
    if(r1 > r2):
        r = r1
    else:
        r = r2
    print(r)
else:
    r = min(X.shape[0], X.shape[1])
    print(r[0])

[1]


In [173]:
np.linalg.matrix_rank(X, 3/np.sqrt(X.shape[0]*X.shape[1]))

4

In [208]:
X

array([[22, 34, 74],
       [53, 64, 96]])

In [209]:
Y = np.array([[3,4,7], [5, 6, 9]])

In [95]:
X[X>10]

array([45, 56])

## Distortion

In [603]:
z = np.sum(X**2, axis = 1)/(2*1e4*1)
y = np.exp((z-1)**2) - 1
idxfind = np.where(z<1)
y[idxfind] = 0
out = sum(y)
print(out)

0.0


In [604]:
def aux_G(X, m0, r):
    z = np.sum(X**2, axis = 1)/(2*m0*r)
    y = np.exp((z-1)**2) - 1
    idxfind = np.where(z<1)
    y[idxfind] = 0
    out = sum(y)
    return out

In [628]:
def aux_F_t(X,Y,S,M_E,E,m0,rho):
    n = X.shape[0]
    r = X.shape[1]
    
    out1 = np.sum((((X@S@(Y.T))-M_E)*E)**2)/2
    out2 = rho*aux_G(Y,m0,r)
    out3 = rho*aux_G(X,m0,r)
    out = out1+out2+out3
    return(out)

In [625]:
np.sum(((X@S@(Y.T))-M_E)*E**2)/2

-0.21367627364940522

In [616]:
Y

array([[-0.20504845,  1.22333936],
       [-0.9274427 ,  0.94661364],
       [-1.44838019, -0.77933525]])

In [629]:
aux_F_t(X,Y,S,M_E,E,m0, rho)

0.09179547747102265

## Gradient

In [587]:
def aux_Gp(X,m0,r):
    z = np.sum(X**2, axis = 1)/(2*m0*r)
    z = 2*np.exp((z-1)**2)/(z-1)
    idxfind = np.where(z<0)
    z[idxfind] = 0
    
    nrow = X.shape[0]
    ncol = X.shape[1]
    
    if nrow < ncol:
        j = 0
        temp_tup = []
        for i in range(ncol):
            temp_tup.append([z[j%4], z[(j+1)%4], z[(j+2)%4]])
            j += 3
    else:
        temp_tup = []
        for i in range(ncol):
            temp_tup.append(np.transpose(z))
            
    z_new = np.transpose(np.array(temp_tup))
    
    
    out = (X*z_new)/(m0*r)
    return out

In [570]:
m0*r

20000.0

In [577]:
z_new = np.array([[0, 0], [0, 0], [0, 0]])

In [557]:
r

2

In [581]:
rho*(X*z_new)/(m0*r)

array([[-0., -0.],
       [-0.,  0.],
       [-0., -0.]])

In [573]:
aux_Gp(X, m0, r)

[1.87596358e-05 7.11250557e-05 6.01153085e-05]
[-5.43646167 -5.43617704 -5.43623688]
(array([0, 1, 2], dtype=int64),)
[0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]]


array([[-0., -0.],
       [-0.,  0.],
       [-0., -0.]])

In [124]:
temp = np.array([1,2,3,4])

In [125]:
temp

array([1, 2, 3, 4])

In [145]:
nrow = 3
ncol = 4

if nrow < ncol:
    j = 0
    temp_tup = []
    for i in range(ncol):
        temp_tup.append([temp[j%4], temp[(j+1)%4], temp[(j+2)%4]])
        j += 3
else:
    temp_tup = []
    for i in range(ncol):
        temp_tup.append(np.transpose(temp))
        

In [146]:
np.transpose(np.array(temp_tup))

array([[1, 4, 3, 2],
       [2, 1, 4, 3],
       [3, 2, 1, 4]])

In [149]:
a = np.array([[1,2],[3,4]])
b = np.array([[2,3],[4,5]])
a@np.transpose(b)

array([[ 8, 14],
       [18, 32]])

In [521]:
M_E

array([[0.49613894, 0.        , 1.48841682],
       [0.99227788, 0.74420841, 0.        ],
       [0.        , 1.73648628, 2.23262522]])

In [522]:
E

array([[1., 0., 1.],
       [1., 1., 0.],
       [0., 1., 1.]])

In [597]:
def aux_gradF_t(X,Y,S,M_E,E,m0,rho):
    n = X.shape[0]
    r = X.shape[1]
    m = Y.shape[0]
    if Y.shape[1] != r:
        raise ValueError("dimension error from aux_gradF_t")
        return 0
    
    XS = (X@S)
    YS = (Y@(S.T))
    XSY = ((XS)@(Y.T))
    
    
    Qx = (((X.T) @ ((M_E - XSY)*E)) @ (YS)/n)
    Qy = (((Y.T) @ (((M_E - XSY)*E).T) @ (XS))/m)
    
    W = (((XSY - M_E)*E)@YS) + (X@Qx) + rho*aux_Gp(X, m0, r)
    Z = ((((XSY - M_E)*E).T)@XS) + (Y@Qy) + rho*aux_Gp(Y, m0, r)
    
    resgrad = {}
    resgrad['W'] = W
    resgrad['Z'] = Z
    
    return resgrad

In [598]:
dist = np.zeros(niter+1)
dist[0] = np.linalg.norm((M_E - ((X)@(S)@(Y.T))*E))/np.sqrt(nnZ_E)

In [481]:
S

array([[ 1.09285922,  0.25504118],
       [-0.24143482,  0.45049079]])

In [562]:
X

array([[-0.72161684, -0.47922288],
       [-0.28566291,  1.66234742],
       [-1.54843981, -0.08334563]])

In [524]:
n

3

In [599]:
# compute the gradient
tmpgrad = aux_gradF_t(X,Y,S,M_E,E,m0, rho)
W = tmpgrad['W']
Z = tmpgrad['Z']

In [600]:
W

array([[ 0.37782701, -0.08125315],
       [ 0.09917488, -0.02132794],
       [-0.19437431,  0.04180094]])

In [601]:
Z

array([[ 0.32656965,  0.0029898 ],
       [-0.3012745 , -0.00275822],
       [ 0.14668265,  0.00134291]])

## Sopt given X & Y

In [211]:
X

array([[22, 34, 74],
       [53, 64, 96]])

In [212]:
Y

array([[3, 4, 7],
       [5, 6, 9]])

In [216]:
X[:,0]

array([22, 53])

In [217]:
Y[:,0]

array([3, 5])

In [218]:
np.outer(X[:,0],Y[:,0])

array([[ 66, 110],
       [159, 265]])

In [234]:
len(X.flatten(order = 'F'))

6

In [229]:
np.reshape(X, (-1,1), order = 'F').shape

(6, 1)

In [236]:
temp = np.zeros((2,2))
temp[:,0] = [1,2]

In [237]:
temp

array([[1., 0.],
       [2., 0.]])

In [241]:
np.matrix([[1,2],[3,4]])*3

matrix([[ 3,  6],
        [ 9, 12]])

In [405]:
r

2

In [515]:
def aux_getoptS(X,Y,M_E,E):
    n = X.shape[0]
    r = X.shape[1]
    
    C = (X.T)@(M_E)@(Y)
    C = np.reshape(C, (-1,1), order = 'F')
    
    nnrow = X.shape[1]*Y.shape[1]
    A = np.zeros((nnrow, r**2))
    
    for i in range(r):
        for j in range(r):
            ind = ((j)*r + i + 1)
            tmp = np.transpose(X)@(np.outer(X[:,i], Y[:,j])*E)@Y
            A[:,ind - 1] = tmp.flatten(order = 'F') # column wise flat karo!!
            
            
    S = np.linalg.solve(A, C)
    out = np.reshape(S, (r, int(S.shape[0]*S.shape[1]/r)), order = 'F')
    return out

## Optimal Line Search

In [245]:
np.linalg.norm(X)

152.43687217992897

In [251]:
np.zeros(21)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [8]:
def aux_getoptT(X,W,Y,Z,S,M_E,E,m0,rho):
    norm2WZ = (np.linalg.norm(W)**2) + (np.linalg.norm(Z)**2)
    f = np.zeros(21)
    f[0] = aux_F_t(X,Y,S,M_E,E,m0,rho)
    t = -1e-1
    for i in range(20):
        f[i+1] = aux_F_t(X+t*W,Y+t*Z,S,M_E,E,m0,rho)
        if((f[i+1]-f[i]) <= 0.5*t*norm2WZ):
            out = t
            break
        t = t/2
    out = t
    return out

In [670]:
# line search for the optimum jump length
t = aux_getoptT(X, W, Y, Z, S, M_E, E, m0, rho)
X = X + t*W
Y = Y + t*Z
S = aux_getoptS(X, Y, M_E, E)

In [673]:
X

array([[-0.79718231, -0.46297224],
       [-0.3054979 ,  1.66661301],
       [-1.50956491, -0.09170583]])

In [672]:
Y

array([[-0.27036244,  1.22274139],
       [-0.86718774,  0.94716529],
       [-1.47771675, -0.77960383]])

In [633]:
t

-0.1

In [671]:
S

array([[ 1.11357722, -0.28413644],
       [ 0.23232554,  0.42990407]])

## OptSpace

In [638]:
niter = 50

In [674]:
A = np.array([[2, np.nan, 6, np.nan], [4, 3, np.nan, np.nan], [np.nan, 7, 9, 4], [np.nan, np.nan, 12, 45]])

In [675]:
A

array([[ 2., nan,  6., nan],
       [ 4.,  3., nan, nan],
       [nan,  7.,  9.,  4.],
       [nan, nan, 12., 45.]])

In [276]:
idxna = np.isnan(A)
M_E = np.zeros((A.shape[0], A.shape[1]))
M_E[idxna == False] = A[idxna == False]

In [277]:
M_E

array([[2., 0., 6.],
       [4., 3., 0.],
       [0., 7., 9.]])

In [288]:
n = A.shape[0]
m = A.shape[1]

In [297]:
nnZ_E = np.sum(idxna == False)
E = np.zeros((A.shape[0], A.shape[1]))
E[idxna == False] = 1
eps = nnZ_E/np.sqrt(m*n)

In [281]:
idxna

array([[False,  True, False],
       [False, False,  True],
       [ True, False, False]])

In [298]:
nnZ_E

6

In [324]:
E

array([[1., 0., 1.],
       [1., 1., 0.],
       [0., 1., 1.]])

In [290]:
eps

2.0

In [293]:
str(3).isnumeric()

True

In [301]:
guess_rank(M_E, nnZ_E)[0]

1

In [302]:
 r = min(max(round(guess_rank(M_E, nnZ_E)[0]), 2), m - 1)

In [303]:
r

2

In [304]:
np.isinf(50)

False

In [318]:
if((np.isinf(niter)) | (niter <= 1) | (str(niter).isnumeric() == False)):
        raise ValueError("OptSpace: invalid niter number")

In [319]:
niter = round(niter)
m0 = 1e4
rho = 0

In [320]:
rescale_param = np.sqrt(nnZ_E*r/(np.linalg.norm(M_E)**2))
M_E = M_E*rescale_param

In [321]:
rescale_param

0.2480694691784169

In [322]:
M_E

array([[0.49613894, 0.        , 1.48841682],
       [0.99227788, 0.74420841, 0.        ],
       [0.        , 1.73648628, 2.23262522]])

In [329]:
M_Et = M_E
d = np.sum(E, axis = 0) ## this is colSums
d_ = np.mean(d)

In [331]:
d

array([2., 2., 2.])

In [330]:
d_

2.0

In [334]:
np.sum(M_E[:,0])

1.4884168150705015

In [346]:
np.where(M_E[:,0]>0)[0]

array([0, 1], dtype=int64)

In [359]:
import random
random.sample(range(2), 2)

[0, 1]

In [368]:
M_Et = M_E
d = np.sum(E, axis = 0)
d_ = np.mean(d)
for col in range(m):
    if(np.sum(E[:,col]) > (2*d_)):
        listed = np.where(E[:,col] > 0)[0]
        p = rm.sample(range(len(listed)), len(listed))
        M_Et[listed[p[np.ceil(2*d_)]]:n, col] = 0

In [370]:
M_Et

array([[0.49613894, 0.        , 1.48841682],
       [0.99227788, 0.74420841, 0.        ],
       [0.        , 1.73648628, 2.23262522]])

In [367]:
np.sum(M_Et, axis = 1) ## this is rowSums

array([1.98455575, 1.73648628, 3.96911151])

In [369]:
d = np.sum(E, axis = 1)
d_ = np.mean(d)
for row in range(n):
    if(np.sum(E[row,:]) > (2*d_)):
        listed = np.where(E[row,:] > 0)[0]
        p = rm.sample(range(len(listed)), len(listed))
        M_Et[row, listed[p[np.ceil(2*d_)]]:m] = 0

In [374]:
temp = True
if(temp):
    print("Its working")

Its working


In [397]:
svdEt = np.linalg.svd(M_Et)
X0 = svdEt[0][:,0:r]
S0 = np.diag(svdEt[1][0:r])
Y0 = svdEt[2][0:r,:].T

In [391]:
X0

array([[-0.41662567, -0.27667946],
       [-0.16492756,  0.95975673],
       [-0.89399214, -0.04811962]])

In [392]:
S0

array([[3.12842749, 0.        ],
       [0.        , 1.15401293]])

In [394]:
svdEt

(array([[-0.41662567, -0.27667946,  0.86595122],
        [-0.16492756,  0.95975673,  0.22730139],
        [-0.89399214, -0.04811962, -0.44549136]]),
 array([3.12842749, 1.15401293, 0.93872019]),
 array([[-0.11838478, -0.53545929, -0.83622269],
        [ 0.70629531,  0.54652764, -0.44994942],
        [ 0.69794841, -0.64388732,  0.31349183]]))

In [398]:
Y0

array([[-0.11838478,  0.70629531],
       [-0.53545929,  0.54652764],
       [-0.83622269, -0.44994942]])

In [399]:
X0 = X0*np.sqrt(n)
Y0 = Y0*np.sqrt(m)
S0 = S0/eps

In [400]:
X0

array([[-0.72161684, -0.47922288],
       [-0.28566291,  1.66234742],
       [-1.54843981, -0.08334563]])

In [401]:
Y0

array([[-0.20504845,  1.22333936],
       [-0.9274427 ,  0.94661364],
       [-1.44838019, -0.77933525]])

In [402]:
S0

array([[1.56421375, 0.        ],
       [0.        , 0.57700646]])

In [514]:
S.T

array([[ 1.09285922, -0.24143482],
       [ 0.25504118,  0.45049079]])

In [516]:
X = X0
Y = Y0
S = aux_getoptS(X, Y, M_E, E)

In [517]:
S

array([[ 1.09285922, -0.24143482],
       [ 0.25504118,  0.45049079]])

In [458]:
S.shape[0]

array([[ 1.09285922,  0.25504118],
       [-0.24143482,  0.45049079]])

In [449]:
S

array([[ 1.09285922],
       [ 0.25504118],
       [-0.24143482],
       [ 0.45049079]])

In [412]:
X.T

array([[-0.72161684, -0.28566291, -1.54843981],
       [-0.47922288,  1.66234742, -0.08334563]])

In [413]:
M_E

array([[0.49613894, 0.        , 1.48841682],
       [0.99227788, 0.74420841, 0.        ],
       [0.        , 1.73648628, 2.23262522]])

In [422]:
Y

array([[-0.20504845,  1.22333936],
       [-0.9274427 ,  0.94661364],
       [-1.44838019, -0.77933525]])

In [426]:
(X.T)@(M_E)

array([[-0.64147919, -2.90143723, -4.53115241],
       [ 1.41174943,  1.09240438, -0.89936296]])

In [434]:
(np.X.T)@(M_E)@(Y)

array([[ 9.38528247e+00, -1.03278295e-15],
       [-8.60056673e-16,  3.46203878e+00]])

In [466]:
for i in range(1,5):
    print(i)

1
2
3
4


In [469]:
temp = [1,2,3,4,5]

In [470]:
temp = temp[1:5]

In [471]:
temp

[2, 3, 4, 5]

In [655]:
dist[i+1] = np.linalg.norm((M_E - X@S@(Y.T))*E)/np.sqrt(nnZ_E)

In [657]:
dist[5]

0.09335584211579384

In [676]:
def OptSpace(A, ropt = None, niter = 50, tol = 1e-6, showprogress = True):
    import random as rd
    ## Preprocessing: A is a partially revealed matrix
    if(A.ndim != 2):
        raise ValueError("Optspace: input A should be a 2-d array")
    if(np.isinf(A).any()):
        raise ValueError("OptSpace: no infinite value in A is allowed")
    if(np.isnan(A).any() == False):
        raise ValueError("OptSpace: there is no unobserved values as NA")
        
    idxna = np.isnan(A)
    M_E = np.zeros((A.shape[0], A.shape[1]))
    M_E[idxna == False] = A[idxna == False] # replacing the null values in A with 0 in M_E
    
    ## Preprocessing: size info
    n = A.shape[0]
    m = A.shape[1]
    
    ## preprocessing: other sparse-related concepts
    nnz_E = np.sum(idxna == False)
    E = np.zeros((A.shape[0], A.shape[1]))
    E[idxna == False] = 1  # replacing the null values in A with 0 in E
    eps = nnz_E/np.sqrt(m*n)
    
    ## Preprocessing: ropt: implied rank
    if(ropt == None):
        if(showprogress):
            print("OptSpace: Guessing an implicit rank...")
        r = min(max(round(guess_rank(M_E, nnZ_E)[0]), 2), m - 1)
        if(showprogress):
            print("OptSpace: Estimated rank: ", r)
    else:
        r = round(ropt)
        if((str(r).isnumeric()) | (r < 1) | (r > m) | (r > n)):
            raise ValueError("OptSPace: ropt should be an interger in {}".format([1, min(A.shape[0], A.shape[1])]))
    
    ## Preprocessing: niter: maximum number of iterations
    if((np.isinf(niter)) | (niter <= 1) | (str(niter).isnumeric() == False)):
        raise ValueError("OptSpace: invalid niter number")
        
    niter = round(niter)
    m0 = 1e4
    rho = 0
    
    ## Main Computation
    rescale_param = np.sqrt(nnZ_E*r/(np.linalg.norm(M_E)**2))
    M_E = M_E*rescale_param
    
    ## 1. Trimming
    if(showprogress):
        print("OptSpace: Step 1: Trimming...")
    
    M_Et = M_E
    
    d = np.sum(E, axis = 0)
    d_ = np.mean(d)
    for col in range(m):
        if(np.sum(E[:,col]) > (2*d_)):
            listed = np.where(E[:,col] > 0)[0]
            p = rm.sample(range(len(listed)), len(listed))
            M_Et[listed[p[np.ceil(2*d_)]]:n, col] = 0
            
    d = np.sum(E, axis = 1)
    d_ = np.mean(d)
    for row in range(n):
        if(np.sum(E[row,:]) > (2*d_)):
            listed = np.where(E[row,:] > 0)[0]
            p = rm.sample(range(len(listed)), len(listed))
            M_Et[row, listed[p[np.ceil(2*d_)]]:m] = 0
            
            
    ## 2. SVD
    if(showprogress):
        print("OptSpace: Step 2: SVD...")
    
    svdEt = np.linalg.svd(M_Et)
    X0 = svdEt[0][:,1:r]
    S0 = np.diag(svdEt[1][1:r])
    Y0 = svdEt[2][:,1:r]
    
    
    ## 3. Initial Guess
    if(showprogress):
        print("OptSpace: Step 3: Initial Guess...")
        
    X0 = X0*np.sqrt(n)
    Y0 = Y0*np.sqrt(m)
    S0 = S0/eps
    
    
    ## 4. Gradient Descent
    if(showprogress):
        print("OptSpace: Step 4: Gradient Descent...")
        
    X = X0
    Y = Y0
    S = aux_getoptS(X, Y, M_E, E)
    
    
    # initialize
    dist = np.zeros(niter+1)
    dist[0] = np.linalg.norm((M_E - ((X)@(S)@(Y.T))*E))/np.sqrt(nnZ_E)
    for i in range(niter):
        # compute the gradient
        tmpgrad = aux_gradF_t(X, Y, S, M_E, E, m0, rho)
        W = tmpgrad['W']
        Z = tmpgrad['Z']
        
        # line search for the optimum jump length
        t = aux_getoptT(X, W, Y, Z, S, M_E, E, m0, rho)
        X = X + t*W
        Y = Y + t*Z
        S = aux_getoptS(X, Y, M_E, E)
        
        # compute distortion
        dist[i+1] = np.linalg.norm((M_E - X@S@(Y.T))*E)/np.sqrt(nnZ_E)
        if(showprogress):
            pmsg = print("OptSpace: Step 4: Iteration {}: distortion: {}".format(i, dist[i+1]))
          
        
        if(dist[i+1] < tol):
            dist = dist[1:(i+1)]
            break
            
        #print(dist)
        
    S = S/rescale_param
            
     # Return Results
    out = dict()
    out['X'] = X
    out['S'] = S
    out['Y'] = Y
    out['dist'] = dist
    
    if(showprogress):
        print("OptSpace: estimation finished.")
    
    return out

In [4]:
A = np.array([[2, np.nan, 6], [4, 3, np.nan], [np.nan, 7, 9], [np.nan, np.nan, 12]])

In [3]:
A

array([[ 2., nan,  6.],
       [ 4.,  3., nan],
       [nan,  7.,  9.],
       [nan, nan, 12.]])

In [687]:
out = OptSpace(A)

OptSpace: Guessing an implicit rank...
OptSpace: Estimated rank:  2
OptSpace: Step 1: Trimming...
OptSpace: Step 2: SVD...
OptSpace: Step 3: Initial Guess...
OptSpace: Step 4: Gradient Descent...
OptSpace: Step 4: Iteration 0: distortion: 1.313954032267814
OptSpace: Step 4: Iteration 1: distortion: 1.3033305782679536
OptSpace: Step 4: Iteration 2: distortion: 1.2913198086428026
OptSpace: Step 4: Iteration 3: distortion: 1.2772702682004025
OptSpace: Step 4: Iteration 4: distortion: 1.2601967803197158
OptSpace: Step 4: Iteration 5: distortion: 1.238641610798357
OptSpace: Step 4: Iteration 6: distortion: 1.2105616966216968
OptSpace: Step 4: Iteration 7: distortion: 1.173499382586156
OptSpace: Step 4: Iteration 8: distortion: 1.125756195680379
OptSpace: Step 4: Iteration 9: distortion: 1.0695282709956804
OptSpace: Step 4: Iteration 10: distortion: 1.014294112178106
OptSpace: Step 4: Iteration 11: distortion: 0.9728943274295448
OptSpace: Step 4: Iteration 12: distortion: 0.9499085291371481


In [688]:
A

array([[ 2., nan,  6.],
       [ 4.,  3., nan],
       [nan,  7.,  9.],
       [nan, nan, 12.]])

In [681]:
out['X']

array([[ 0.66992175],
       [ 0.19844283],
       [-0.27663049],
       [-1.93353821]])

In [684]:
out['S']

array([[-11.79043806]])

In [689]:
out['X']@out['S']@(out['Y'].T)

array([[  0.75127926, -15.69380877,   5.8992627 ],
       [ -0.15341164,   3.20468433,  -1.20463267],
       [ -0.14705967,   3.07199536,  -1.15475522],
       [  1.53434562, -32.05163242,  12.04812689]])

In [1]:
%load_ext autoreload
%autoreload 2

In [62]:
from OptSpace import OptSpaceImputer

In [63]:
opt = OptSpaceImputer(verbose = True)

In [44]:
A

array([[ 2., nan,  6.],
       [ 4.,  3., nan],
       [nan,  7.,  9.],
       [nan, nan, 12.]])

In [64]:
A_imp = opt.fit_transform(A)

OptSpace: Guessing an implicit rank...
OptSpace: Estimated rank:  2
OptSpace: Step 1: Trimming...
OptSpace: Step 2: SVD...
OptSpace: Step 3: Initial Guess...
OptSpace: Step 4: Gradient Descent...
OptSpace: Step 4: Iteration 0: distortion: 1.3123029261742096
OptSpace: Step 4: Iteration 1: distortion: 1.2996417202844839
OptSpace: Step 4: Iteration 2: distortion: 1.2849116027136622
OptSpace: Step 4: Iteration 3: distortion: 1.2669395088780953
OptSpace: Step 4: Iteration 4: distortion: 1.2438571554994848
OptSpace: Step 4: Iteration 5: distortion: 1.2128335180703784
OptSpace: Step 4: Iteration 6: distortion: 1.1701932187056039
OptSpace: Step 4: Iteration 7: distortion: 1.1134097402226877
OptSpace: Step 4: Iteration 8: distortion: 1.0472364876587898
OptSpace: Step 4: Iteration 9: distortion: 0.9888442217604099
OptSpace: Step 4: Iteration 10: distortion: 0.9543295267762519
OptSpace: Step 4: Iteration 11: distortion: 0.9402398150388841
OptSpace: Step 4: Iteration 12: distortion: 0.934532167191

In [65]:
A_imp

array([[ 0.68245665, 15.05311887,  5.88885865],
       [ 0.14595227,  3.21930618,  1.25940938],
       [ 0.13143268,  2.89904386,  1.13412109],
       [ 1.39680857, 30.80975983, 12.05293882]])

In [11]:
res['X']@res['S']@(res['Y'].T)

array([[  0.68245665, -15.05311887,   5.88885865],
       [ -0.14595227,   3.21930618,  -1.25940938],
       [ -0.13143268,   2.89904386,  -1.13412109],
       [  1.39680857, -30.80975983,  12.05293882]])